In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx'
df_glu1 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/18_Glu_A.xlsx'
df_glu1_alkb = load_data(glu_path)

In [ ]:
df_glu1 = df_glu1[df_glu1.Mass>1500]
df_glu1_alkb = df_glu1_alkb[df_glu1_alkb.Mass>1500]

In [ ]:
glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_5_rosa.xlsx'
df_glu5 = load_data(glu_path)
glu_path = '/Users/xyuan/Documents/ChenLab/8May2021/UC_glu_6_rosa.xlsx'
df_glu5_alkb = load_data(glu_path)
df_glu5.shape, df_glu5_alkb.shape

In [ ]:
df_glu5 = df_glu5[df_glu5.Mass>1500]
df_glu5_alkb = df_glu5_alkb[df_glu5_alkb.Mass>1500]

In [ ]:
plotly_zone(df_glu5)

In [ ]:
_, df_alkb1_only = diff_dfs(df_glu1, df_glu1_alkb)
df_alkb1_only.shape

In [ ]:
_, df_alkb5_only = diff_dfs(df_glu5, df_glu5_alkb)
df_alkb5_only.shape

In [ ]:
_, df_alkb5_comm = comm_dfs(df_glu5, df_glu5_alkb)
df_alkb5_comm.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16
params = [(df_glu1, df_alkb1_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_sh1.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

In [ ]:
# df_sh1 = df_shift_hits.copy()
df_sh5 = df_shift_hits.copy()

In [ ]:
dfm_l, dfm_r = peer_dfs(df_glu5, df_alkb5_only, shift=1)
dfm_l.shape, dfm_r.shape

In [ ]:
shift = 1
# dfm_l = match_dfs(df_glu1, df_alkb1_only, shift=shift)
# dfm_r = match_dfs(df_alkb1_only, df_glu1, shift=-1*shift)
dfm_l, dfm_r = peer_dfs(df_glu1, df_alkb1_only, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
shift = 1
dfm_wild_l = match_dfs(df_glu1, df_glu1, shift=-1*shift)
dfm_wild_r = match_dfs(df_glu1, df_glu1, shift=shift)

dfm_alkb_l = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=-1*shift)
dfm_alkb_r = match_dfs(df_glu1_alkb, df_glu1_alkb, shift=shift)
dfm_wild_l.shape, dfm_wild_r.shape, dfm_alkb_l.shape, dfm_alkb_r.shape

In [ ]:
df_glu5_extended = pd.concat([df_glu5, dfm_l]).drop_duplicates()
df_glu5_extended.shape

In [ ]:
bcr = base_calling_random(dfm_l, base_only=True)
plotly_basecalling(*bcr)
bcr = base_calling_random(dfm_r, base_only=True)
plotly_basecalling(*bcr)

In [ ]:
plotly_basecalling(*bcr_ext)

In [ ]:
# plotly_zone(dfm_l)
# plotly_zone(dfm_r)
bcr_l = base_calling_random(dfm_l, base_only=True)
bcr_r = base_calling_random(dfm_r, base_only=True)

In [ ]:
plotly_basecalling(*bcr_l)
plotly_basecalling(*bcr_r)